In [3]:
import pandas as pd
import mysql.connector as connection
from google.cloud import bigquery
from dotenv import load_dotenv
import os
load_dotenv()

True

Hacemos una funcion la cual me devolverá los "order_customer_id" que cumplen ciertos requisitos:

In [6]:
def listaOrders(order_customer_id):
    mydb = connection.connect(
        host= os.getenv('DB_HOST'), 
        user= os.getenv('DB_USER'), 
        passwd= os.getenv('DB_PASS'), 
        database=os.getenv('DB_DB') 
    )

    sql = f"SELECT * FROM retail_db.orders WHERE order_customer_id ={order_customer_id}"
    df = pd.read_sql(sql, mydb)
    mydb.close()
    return df.to_json(orient="records") # como lo vamos a aplicar en cloud functions el formato que necesitamos es json (http) sino solo retornamos df

In [7]:
listaOrders(1117)

C:\Users\GGudino\AppData\Local\Temp\ipykernel_30320\2899773967.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, mydb)


'[{"order_id":7565,"order_date":1378684800000,"order_customer_id":1117,"order_status":"ON_HOLD"},{"order_id":8405,"order_date":1379203200000,"order_customer_id":1117,"order_status":"PENDING_PAYMENT"},{"order_id":24437,"order_date":1387756800000,"order_customer_id":1117,"order_status":"PENDING"},{"order_id":25942,"order_date":1388534400000,"order_customer_id":1117,"order_status":"PROCESSING"},{"order_id":43297,"order_date":1397865600000,"order_customer_id":1117,"order_status":"PENDING_PAYMENT"},{"order_id":68880,"order_date":1405209600000,"order_customer_id":1117,"order_status":"COMPLETE"}]'

Esta función la llevamos a Cloud Functions en GCP. Para ello deberemos:
1. Hacer click en Crear función
2. Elegir el entorno de 1era generación para generar menos costos
3. Darle un nombre a la función
4. 